In [1]:
from constants.abs_paths import AbsDirPath
from processing_pipeline.utilities.data_transformation import load_all_files

In [2]:
from processing_pipeline.data_optimization.stage_cfg import S0NoiseFiltering

current_stage_cfg = S0NoiseFiltering

In [14]:
current_stage_cfg = current_stage_cfg.next_stage
current_stage_cfg

StageConfig(name='s3', depends_on_fields=['sentence'], resulting_fields=['tactic', 'response'], data_dir=WindowsPath('C:/Users/Dmitrijs/Documents/myDocs/masters/courses/thesis/code/data/keywords/s3_tactic_extraction'), boolean_field_name=None, next_stage=None)

In [9]:
in_dir = current_stage_cfg.data_dir
out_path = AbsDirPath.SAMPLES / (current_stage_cfg.name + ".csv")
in_dir, out_path

(WindowsPath('C:/Users/Dmitrijs/Documents/myDocs/masters/courses/thesis/code/data/keywords/s2_arch_relevance_check'),
 WindowsPath('C:/Users/Dmitrijs/Documents/myDocs/masters/courses/thesis/code/data/samples/s2.csv'))

In [11]:
df = load_all_files(in_dir)

Loaded C:\Users\Dmitrijs\Documents\myDocs\masters\courses\thesis\code\data\keywords\s2_arch_relevance_check\allenai.scispacy.v0.5.5.code_comment.parquet
Loaded C:\Users\Dmitrijs\Documents\myDocs\masters\courses\thesis\code\data\keywords\s2_arch_relevance_check\allenai.scispacy.v0.5.5.docs.parquet
Loaded C:\Users\Dmitrijs\Documents\myDocs\masters\courses\thesis\code\data\keywords\s2_arch_relevance_check\allenai.scispacy.v0.5.5.issue.parquet
Loaded C:\Users\Dmitrijs\Documents\myDocs\masters\courses\thesis\code\data\keywords\s2_arch_relevance_check\allenai.scispacy.v0.5.5.issue_comment.parquet
Loaded C:\Users\Dmitrijs\Documents\myDocs\masters\courses\thesis\code\data\keywords\s2_arch_relevance_check\allenai.scispacy.v0.5.5.release.parquet
Loaded C:\Users\Dmitrijs\Documents\myDocs\masters\courses\thesis\code\data\keywords\s2_arch_relevance_check\google.deepvariant.v1.6.1.code_comment.parquet
Loaded C:\Users\Dmitrijs\Documents\myDocs\masters\courses\thesis\code\data\keywords\s2_arch_relevan

In [12]:
sample_size = 384

In [13]:
out_path.parent.mkdir(exist_ok=True)
df.sample(sample_size).to_csv(out_path)
